In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
import warnings
warnings.filterwarnings('ignore')



In [ ]:
# ============================================================================
# PHẦN 1: MODEL DEFINITION - Tự định nghĩa kiến trúc ProphetNet
# ============================================================================

class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention mechanism chuẩn cho Transformer.
    Tự implement từ đầu để hiểu rõ cấu trúc.
    """
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0, f"d_model ({d_model}) phải chia hết cho num_heads ({num_heads})"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Linear layers cho Q, K, V
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        # Linear projections và reshape thành (batch, num_heads, seq_len, d_k)
        Q = self.q_linear(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.k_linear(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.v_linear(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            # Expand mask nếu cần
            if mask.dim() == 2:
                mask = mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
            
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Apply attention to values
        context = torch.matmul(attention_weights, V)
        
        # Reshape và output projection
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.out_linear(context)
        
        return output, attention_weights


class FeedForward(nn.Module):
    """
    Position-wise Feed-Forward Network.
    FFN(x) = max(0, xW1 + b1)W2 + b2
    """
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.linear2(self.dropout(F.relu(self.linear1(x))))


class ProphetNetEncoderLayer(nn.Module):
    """
    Một layer của ProphetNet Encoder.
    Gồm: Self-Attention + FFN + Layer Norm + Residual Connection
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention với residual connection
        attn_output, _ = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed-forward với residual connection
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x


class ProphetNetDecoderLayer(nn.Module):
    """
    Một layer của ProphetNet Decoder với khả năng dự đoán n-future tokens.
    
    Đặc biệt của ProphetNet:
    - Decoder có thêm mechanism để dự đoán nhiều token tương lai cùng lúc
    - Ngoài main stream (dự đoán token tiếp theo), còn có n-stream predicting
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        
        # Self-attention trong decoder (masked)
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        
        # Cross-attention với encoder outputs
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        
        # N-stream self-attention cho future prediction (đặc trưng của ProphetNet)
        self.ngram_self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.norm4 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, self_attn_mask=None, cross_attn_mask=None):
        # 1. Masked self-attention
        attn_output, _ = self.self_attn(x, x, x, self_attn_mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # 2. N-gram self-attention (đặc trưng ProphetNet - dự đoán future tokens)
        ngram_output, _ = self.ngram_self_attn(x, x, x, self_attn_mask)
        x = self.norm2(x + self.dropout(ngram_output))
        
        # 3. Cross-attention với encoder
        cross_output, _ = self.cross_attn(x, encoder_output, encoder_output, cross_attn_mask)
        x = self.norm3(x + self.dropout(cross_output))
        
        # 4. Feed-forward
        ff_output = self.feed_forward(x)
        x = self.norm4(x + self.dropout(ff_output))
        
        return x


class PositionalEncoding(nn.Module):
    """Positional Encoding để mô hình hiểu được thứ tự các token"""
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class ProphetNetModel(nn.Module):
    """
    ProphetNet Model hoàn chỉnh: Encoder-Decoder architecture
    
    Đặc điểm:
    - Encoder: Transformer Encoder chuẩn
    - Decoder: Mở rộng với n-stream prediction (dự đoán nhiều future tokens)
    - Có thể dự đoán ngram_size token tương lai cùng lúc
    """
    def __init__(
        self, 
        vocab_size=30522,  # Kích thước vocabulary
        d_model=1024,       # Dimension của model
        num_encoder_layers=12,
        num_decoder_layers=12,
        num_heads=16,
        d_ff=4096,
        dropout=0.1,
        max_position_embeddings=512,
        ngram_size=2  # Số future tokens dự đoán cùng lúc (đặc trưng ProphetNet)
    ):
        super().__init__()
        
        self.d_model = d_model
        self.ngram_size = ngram_size
        self.vocab_size = vocab_size
        
        # Embedding layers
        self.encoder_embed = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.decoder_embed = nn.Embedding(vocab_size, d_model, padding_idx=0)
        
        # Positional encoding
        self.pos_encoding = PositionalEncoding(d_model, max_position_embeddings)
        
        # Encoder layers
        self.encoder_layers = nn.ModuleList([
            ProphetNetEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_encoder_layers)
        ])
        
        # Decoder layers
        self.decoder_layers = nn.ModuleList([
            ProphetNetDecoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_decoder_layers)
        ])
        
        # Output projection heads
        # Main stream: dự đoán token tiếp theo
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        # N-gram streams: dự đoán n future tokens (đặc trưng ProphetNet)
        self.ngram_heads = nn.ModuleList([
            nn.Linear(d_model, vocab_size, bias=False)
            for _ in range(ngram_size)
        ])
        
        self.dropout = nn.Dropout(dropout)
        
        self._init_weights()
        
    def _init_weights(self):
        """Khởi tạo trọng số theo distribution chuẩn"""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
    def encode(self, input_ids, attention_mask=None):
        """
        Encoder forward pass
        Args:
            input_ids: (batch_size, src_len)
            attention_mask: (batch_size, src_len)
        Returns:
            encoder_output: (batch_size, src_len, d_model)
        """
        # Embedding + Positional Encoding
        x = self.encoder_embed(input_ids) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # Qua các encoder layers
        for layer in self.encoder_layers:
            x = layer(x, attention_mask)
            
        return x
    
    def decode(self, decoder_input_ids, encoder_output, 
               decoder_attention_mask=None, encoder_attention_mask=None):
        """
        Decoder forward pass với n-stream prediction
        Args:
            decoder_input_ids: (batch_size, tgt_len)
            encoder_output: (batch_size, src_len, d_model)
        Returns:
            main_logits: (batch_size, tgt_len, vocab_size)
            ngram_logits: list of (batch_size, tgt_len, vocab_size)
        """
        batch_size, tgt_len = decoder_input_ids.shape
        
        # Embedding + Positional Encoding
        x = self.decoder_embed(decoder_input_ids) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # Tạo causal mask (decoder chỉ nhìn thấy các token trước đó)
        causal_mask = torch.tril(torch.ones(tgt_len, tgt_len, device=x.device))
        causal_mask = causal_mask.unsqueeze(0).unsqueeze(0)
        
        # Qua các decoder layers
        for layer in self.decoder_layers:
            x = layer(x, encoder_output, causal_mask, encoder_attention_mask)
        
        # Main stream prediction (token tiếp theo)
        main_logits = self.lm_head(x)
        
        # N-gram stream predictions (future tokens)
        ngram_logits = [head(x) for head in self.ngram_heads]
        
        return main_logits, ngram_logits
    
    def forward(self, input_ids, decoder_input_ids, 
                attention_mask=None, decoder_attention_mask=None):
        """
        Full forward pass
        Returns:
            main_logits: Dự đoán token tiếp theo
            ngram_logits: Dự đoán n future tokens (đặc trưng ProphetNet)
        """
        encoder_output = self.encode(input_ids, attention_mask)
        main_logits, ngram_logits = self.decode(
            decoder_input_ids, encoder_output, 
            decoder_attention_mask, attention_mask
        )
        return main_logits, ngram_logits


# ============================================================================
# PHẦN 2: LOAD PRETRAINED WEIGHTS - Load trọng số từ Hugging Face
# ============================================================================

def load_pretrained_prophetnet_weights(model, pretrained_model_name='microsoft/prophetnet-large-uncased'):
    """
    Load pretrained weights từ Hugging Face model vào custom ProphetNet model.
    
    Lưu ý: Việc mapping weights từ Hugging Face implementation sang custom
    implementation có thể cần điều chỉnh tên các layers để match.
    
    Args:
        model: Custom ProphetNetModel instance
        pretrained_model_name: Tên model trên Hugging Face
    """
    try:
        from transformers import ProphetNetForConditionalGeneration
        
        print(f"Đang load pretrained weights từ {pretrained_model_name}...")
        
        # Load pretrained model từ Hugging Face
        hf_model = ProphetNetForConditionalGeneration.from_pretrained(pretrained_model_name)
        hf_state_dict = hf_model.state_dict()
        
        # Custom model state dict
        custom_state_dict = model.state_dict()
        
        # Mapping và load weights (simplified version)
        loaded_keys = []
        not_loaded = []
        
        for key in custom_state_dict.keys():
            # Try exact match first
            if key in hf_state_dict and custom_state_dict[key].shape == hf_state_dict[key].shape:
                custom_state_dict[key] = hf_state_dict[key]
                loaded_keys.append(key)
            else:
                not_loaded.append(key)
        
        # Load weights
        model.load_state_dict(custom_state_dict, strict=False)
        
        print(f"✓ Đã load {len(loaded_keys)}/{len(custom_state_dict)} layers từ pretrained model")
        if len(not_loaded) > 0 and len(not_loaded) < 10:
            print(f"  Không load được: {len(not_loaded)} layers")
        
        return model
        
    except ImportError:
        print("⚠ Warning: transformers library chưa được cài đặt.")
        print("  Để load pretrained weights: pip install transformers")
        return model
    except Exception as e:
        print(f"⚠ Warning: Không thể load pretrained weights: {str(e)[:100]}")
        print("  Model sẽ sử dụng random initialization")
        return model


# ============================================================================
# PHẦN 3: FINE-TUNING EXAMPLE - Minh họa fine-tune trên custom data
# ============================================================================

class SimpleSeq2SeqDataset(Dataset):
    """
    Dataset đơn giản cho sequence-to-sequence tasks
    Ví dụ: Summarization hoặc Translation
    """
    def __init__(self, source_texts, target_texts, vocab_size=30522, max_length=128):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.vocab_size = vocab_size
        self.max_length = max_length
        
    def __len__(self):
        return len(self.source_texts)
    
    def __getitem__(self, idx):
        # Trong thực tế, cần tokenize với tokenizer thật
        # Đây chỉ là mock để minh họa
        src = self.simple_tokenize(self.source_texts[idx])
        tgt = self.simple_tokenize(self.target_texts[idx])
        return src, tgt
    
    def simple_tokenize(self, text):
        """Mock tokenization - thực tế cần dùng tokenizer thật"""
        # Giả lập token ids (random cho demo nhưng consistent)
        torch.manual_seed(hash(text) % 2**32)  # Consistent seed cho cùng text
        length = min(len(text.split()), self.max_length)
        tokens = torch.randint(1, min(1000, self.vocab_size), (length,))
        
        # Pad nếu cần
        if len(tokens) < self.max_length:
            tokens = F.pad(tokens, (0, self.max_length - len(tokens)), value=0)
        
        return tokens


def compute_prophetnet_loss(main_logits, ngram_logits, target_ids, ngram_size=2):
    """
    Tính loss cho ProphetNet với main stream + n-gram streams
    
    Args:
        main_logits: (batch_size, seq_len, vocab_size) - dự đoán token tiếp theo
        ngram_logits: list of (batch_size, seq_len, vocab_size) - dự đoán future tokens
        target_ids: (batch_size, seq_len) - ground truth labels
        ngram_size: số future tokens dự đoán
    """
    batch_size = main_logits.size(0)
    seq_len = main_logits.size(1)
    vocab_size = main_logits.size(-1)
    
    # Main stream loss (dự đoán token t+1)
    # main_logits có shape (batch, seq_len_decoder, vocab)
    # target_ids có shape (batch, seq_len_target)
    # Decoder input được shift right, nên cần align với target
    
    # Lấy min length để đảm bảo không bị index out of range
    min_len = min(seq_len, target_ids.size(1) - 1)
    
    if min_len > 0:
        # Dự đoán từ logits[0:min_len] cho target[1:min_len+1]
        main_loss = F.cross_entropy(
            main_logits[:, :min_len].reshape(-1, vocab_size),
            target_ids[:, 1:min_len+1].reshape(-1),
            ignore_index=0  # Bỏ qua padding tokens
        )
    else:
        main_loss = torch.tensor(0.0, device=main_logits.device)
    
    # N-gram losses (dự đoán token t+2, t+3, ..., t+n)
    ngram_loss = torch.tensor(0.0, device=main_logits.device)
    ngram_count = 0
    
    for i, logits in enumerate(ngram_logits):
        shift = i + 2  # t+2, t+3, ...
        # Đảm bảo có đủ target tokens để shift
        if shift < target_ids.size(1):
            # Tính toán valid length
            valid_len = min(logits.size(1), target_ids.size(1) - shift)
            
            if valid_len > 0:
                loss_i = F.cross_entropy(
                    logits[:, :valid_len].reshape(-1, vocab_size),
                    target_ids[:, shift:shift+valid_len].reshape(-1),
                    ignore_index=0
                )
                ngram_loss += loss_i
                ngram_count += 1
    
    # Tổng loss (có thể điều chỉnh weight)
    if ngram_count > 0:
        total_loss = main_loss + 0.5 * (ngram_loss / ngram_count)
    else:
        total_loss = main_loss
    
    return total_loss


def fine_tune_prophetnet(model, train_data, num_epochs=3, learning_rate=5e-5, batch_size=4):
    """
    Fine-tune ProphetNet trên custom dataset
    
    Args:
        model: ProphetNetModel instance
        train_data: Tuple (source_texts, target_texts)
        num_epochs: Số epochs training
        learning_rate: Learning rate
        batch_size: Batch size
    """
    print("\n" + "="*70)
    print("BẮT ĐẦU FINE-TUNING")
    print("="*70)
    
    source_texts, target_texts = train_data
    
    # Tạo dataset và dataloader
    dataset = SimpleSeq2SeqDataset(source_texts, target_texts, 
                                    vocab_size=model.vocab_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Set model to training mode
    model.train()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    print(f"Device: {device}")
    print(f"Training samples: {len(dataset)}")
    print(f"Batch size: {batch_size}")
    print(f"Learning rate: {learning_rate}")
    print()
    
    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0
        
        for batch_idx, (src_batch, tgt_batch) in enumerate(dataloader):
            src_batch = src_batch.to(device)
            tgt_batch = tgt_batch.to(device)
            
            # Forward pass
            # Decoder input = target shifted right (teacher forcing)
            decoder_input = tgt_batch[:, :-1]
            
            try:
                main_logits, ngram_logits = model(
                    input_ids=src_batch,
                    decoder_input_ids=decoder_input
                )
                
                # Compute loss
                loss = compute_prophetnet_loss(main_logits, ngram_logits, tgt_batch)
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                
                # Gradient clipping để tránh exploding gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                
                total_loss += loss.item()
                num_batches += 1
                
                if (batch_idx + 1) % 10 == 0:
                    print(f"Epoch [{epoch+1}/{num_epochs}], "
                          f"Batch [{batch_idx+1}/{len(dataloader)}], "
                          f"Loss: {loss.item():.4f}")
                          
            except RuntimeError as e:
                print(f"⚠ Lỗi tại batch {batch_idx+1}: {str(e)[:100]}")
                continue
        
        if num_batches > 0:
            avg_loss = total_loss / num_batches
            print(f"\n→ Epoch {epoch+1} - Average Loss: {avg_loss:.4f}\n")
        else:
            print(f"\n⚠ Epoch {epoch+1} - Không có batch nào hoàn thành\n")
    
    print("="*70)
    print("HOÀN THÀNH FINE-TUNING")
    print("="*70)
    
    return model


# ============================================================================
# MAIN - Ví dụ sử dụng đầy đủ
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("PROPHETNET IMPLEMENTATION - TỰ ĐỊNH NGHĨA KIẾN TRÚC")
    print("="*70)
    print()
    
    # 1. TẠO MODEL (tự định nghĩa kiến trúc)
    print("BƯỚC 1: Khởi tạo ProphetNet model với custom architecture")
    print("-" * 70)
    
    model = ProphetNetModel(
        vocab_size=30522,
        d_model=512,  # Giảm size cho demo nhanh hơn
        num_encoder_layers=6,
        num_decoder_layers=6,
        num_heads=8,
        d_ff=2048,
        dropout=0.1,
        ngram_size=2  # Dự đoán 2 future tokens
    )
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"✓ Model đã được khởi tạo với:")
    print(f"  - Encoder layers: 6")
    print(f"  - Decoder layers: 6")
    print(f"  - Hidden size: 512")
    print(f"  - Attention heads: 8")
    print(f"  - N-gram prediction: 2 future tokens")
    print(f"  - Tổng parameters: {total_params:,}")
    print(f"  - Trainable parameters: {trainable_params:,}")
    print()
    
    # 2. LOAD PRETRAINED WEIGHTS (optional)
    print("\nBƯỚC 2: Load pretrained weights từ Hugging Face")
    print("-" * 70)
    print("Note: Bước này optional, có thể skip nếu muốn train from scratch")
    # Uncomment dòng dưới để load pretrained weights
    # model = load_pretrained_prophetnet_weights(model)
    print("⊗ Skipped - Sử dụng random initialization cho demo")
    print()
    
    # 3. FINE-TUNE trên custom data
    print("\nBƯỚC 3: Fine-tune model trên custom dataset")
    print("-" * 70)
    
    # Mock data cho demo (thay bằng data thật trong thực tế)
    source_texts = [
        "This is a long document that needs to be summarized for quick reading.",
        "Machine learning models require large amounts of training data to perform well.",
        "ProphetNet can predict multiple future tokens simultaneously during decoding.",
        "Natural language processing has made significant progress in recent years.",
        "Deep learning architectures continue to improve performance on various tasks.",
    ] * 6  # Nhân lên để có nhiều samples hơn
    
    target_texts = [
        "Document summarization for quick reading.",
        "ML models need training data.",
        "ProphetNet predicts multiple future tokens.",
        "NLP has progressed significantly.",
        "Deep learning improves task performance.",
    ] * 6
    
    # Fine-tune model
    try:
        model = fine_tune_prophetnet(
            model=model,
            train_data=(source_texts, target_texts),
            num_epochs=2,
            learning_rate=5e-5,
            batch_size=4
        )
        
        print("\n" + "="*70)
        print("✓ HOÀN TẤT THÀNH CÔNG!")
        print("="*70)
        
    except Exception as e:
        print("\n" + "="*70)
        print(f"⚠ LỖI: {str(e)}")
        print("="*70)
    
    print("\nTÓM TẮT:")
    print("1. ✓ Đã tự định nghĩa kiến trúc ProphetNet từ đầu")
    print("2. ✓ Có thể load pretrained weights từ Hugging Face (nếu cần)")
    print("3. ✓ Đã fine-tune model trên custom dataset")
    print("\nĐặc điểm ProphetNet:")
    print("• Encoder: Transformer Encoder chuẩn")
    print("• Decoder: Mở rộng với n-stream prediction")
    print("• Có thể dự đoán nhiều future tokens cùng lúc")
    print("• Sử dụng main stream loss + n-gram stream losses")
    print("\n" + "="*70)

PROPHETNET IMPLEMENTATION - TỰ ĐỊNH NGHĨA KIẾN TRÚC

BƯỚC 1: Khởi tạo ProphetNet model với custom architecture
----------------------------------------------------------------------
✓ Model đã được khởi tạo với:
  - Encoder layers: 6
  - Decoder layers: 6
  - Hidden size: 512
  - Attention heads: 8
  - N-gram prediction: 2 future tokens
  - Tổng parameters: 128,584,704
  - Trainable parameters: 128,584,704


BƯỚC 2: Load pretrained weights từ Hugging Face
----------------------------------------------------------------------
Note: Bước này optional, có thể skip nếu muốn train from scratch
⊗ Skipped - Sử dụng random initialization cho demo


BƯỚC 3: Fine-tune model trên custom dataset
----------------------------------------------------------------------

BẮT ĐẦU FINE-TUNING
Device: cpu
Training samples: 30
Batch size: 4
Learning rate: 5e-05


→ Epoch 1 - Average Loss: 15.1134


→ Epoch 2 - Average Loss: 14.4609

HOÀN THÀNH FINE-TUNING

✓ HOÀN TẤT THÀNH CÔNG!

TÓM TẮT:
1. ✓ Đã tự định n